## Read data from train.txt and filter it from unwanted patterns


In [14]:
import numpy as np
from keras.utils import to_categorical
from Preprocessing import utils, character_encoding
from Models import rnn
# from CharacterEmbeddings import TF_IDF, WordToVec

## Preprocessing


In [15]:
training_set = utils.read_data(f"./Dataset/train.txt")
print("training_set", training_set[0:500])
filtered_training_set = utils.filter_data(training_set)
print("filtered_training_set", filtered_training_set[0:500])


training_set قَوْلُهُ : ( أَوْ قَطَعَ الْأَوَّلُ يَدَهُ إلَخْ ) قَالَ الزَّرْكَشِيُّ( 14 / 123 )
ابْنُ عَرَفَةَ : قَوْلُهُ : بِلَفْظٍ يَقْتَضِيه كَإِنْكَارِ غَيْرِ حَدِيثٍ بِالْإِسْلَامِ وُجُوبَ مَا عُلِمَ وُجُوبُهُ مِنْ الدِّينِ ضَرُورَةً ( كَإِلْقَاءِ مُصْحَفٍ بِقَذَرٍ وَشَدِّ زُنَّارٍ ) ابْنُ عَرَفَةَ : قَوْلُ ابْنِ شَاسٍ : أَوْ بِفِعْلٍ يَتَضَمَّنُهُ هُوَ كَلُبْسِ الزُّنَّارِ وَإِلْقَاءِ الْمُصْحَفِ فِي صَرِيحِ النَّجَاسَةِ وَالسُّجُودِ لِلصَّنَمِ وَنَحْوِ ذَلِكَ ( وَسِحْرٍ ) مُحَمَّدٌ : قَوْلُ مَالِكٍ و
filtered_training_set قَوْلُهُ أَوْ قَطَعَ الْأَوَّلُ يَدَهُ إلَخْ قَالَ الزَّرْكَشِيُّ 
ابْنُ عَرَفَةَ قَوْلُهُ بِلَفْظٍ يَقْتَضِيه كَإِنْكَارِ غَيْرِ حَدِيثٍ بِالْإِسْلَامِ وُجُوبَ مَا عُلِمَ وُجُوبُهُ مِنْ الدِّينِ ضَرُورَةً كَإِلْقَاءِ مُصْحَفٍ بِقَذَرٍ وَشَدِّ زُنَّارٍ ابْنُ عَرَفَةَ قَوْلُ ابْنِ شَاسٍ أَوْ بِفِعْلٍ يَتَضَمَّنُهُ هُوَ كَلُبْسِ الزُّنَّارِ وَإِلْقَاءِ الْمُصْحَفِ فِي صَرِيحِ النَّجَاسَةِ وَالسُّجُودِ لِلصَّنَمِ وَنَحْوِ ذَلِكَ وَسِحْرٍ مُحَمَّدٌ قَوْلُ مَالِكٍ 

In [16]:
words_set = utils.split_data_to_words(filtered_training_set)
text_without_diacritics = []
diacritic_list = []
# Preparing Training Set
text_without_diacritics, diacritic_list = character_encoding.PrepareTrainData(words_set[0:10000])

# Assume this is a test set
text_without_diacritics_test = character_encoding.PrepareTestData(words_set[10000:15000])

In [17]:
# chracter_set = utils.char_tokenizer(filtered_training_set)
# print("chracter_set", chracter_set[0:10])

## Feature Extraction


## Building The Model


In [18]:
output_size = len(character_encoding.DIACRITICS)

In [19]:
# Create an instance of the LSTM class
model = rnn.RNN(input_shape=(None, 1), output_shape = output_size)

In [20]:
# Convert the training data to the required format
X_train = np.array([[[ord(char)]] for word in text_without_diacritics for char in word])
y_train = []
for word_diacritic in diacritic_list:
    for diacritic in word_diacritic:
        #print(utils.map_text_to_diacritic(diacritic))
        index = character_encoding.DIACRITICS.index(diacritic)
        y_train.append(to_categorical(index, num_classes=output_size))
y_train = np.array(y_train)

In [21]:
# Train the model
model.train(X_train, y_train, epochs=5, batch_size=64)

Epoch 1/5
601/601 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.3327 - loss: 1.9392
Epoch 2/5
601/601 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3549 - loss: 1.7807
Epoch 3/5
601/601 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3482 - loss: 1.7781
Epoch 4/5
601/601 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3567 - loss: 1.7652
Epoch 5/5
601/601 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3561 - loss: 1.7622


In [22]:
# Convert the test data to the required format
X_test = np.array([[[ord(char)]] for word in text_without_diacritics_test for char in word])

# Predict the diacritics of the test data
y_pred = model.predict(X_test)

594/594 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step 


## Model Evaluation


In [23]:
# Evaluate the model
lost , accuracy = model.evaluate(X_test, y_pred)

594/594 ━━━━━━━━━━━━━━━━━━━━ 1s 976us/step - accuracy: 1.0000 - loss: 1.7615
